In [ ]:
#r "nuget: Microsoft.DotNet.Interactive.SqlServer, *-*"

In [ ]:
#!connect mssql --kernel-name sql2025 --connection-string "Server=sql-ctp20;TrustServerCertificate=True;Integrated Security=True"

In [ ]:
SELECT @@VERSION

In [ ]:
USE master
IF EXISTS (SELECT name FROM sys.databases WHERE name='CPUvsGPU')BEGIN
    ALTER DATABASE CPUvsGPU SET SINGLE_USER WITH ROLLBACK IMMEDIATE;
    DROP DATABASE CPUvsGPU;
END
GO
CREATE DATABASE CPUvsGPU
GO
USE CPUvsGPU
GO
create master key encryption by password = 'MyTest!Mast3rP4ss'
GO
sp_configure 'external rest endpoint enabled', 1;
RECONFIGURE WITH OVERRIDE

In [ ]:
CREATE EXTERNAL MODEL ollama
WITH (
    LOCATION = 'https://ai-gpu.lab.bwdemo.io:443/api/embed',
    API_FORMAT = 'ollama',
    MODEL_TYPE = EMBEDDINGS,
    MODEL = 'nomic-embed-text'
);

In [ ]:
CREATE EXTERNAL MODEL ollamacpu
WITH (
    LOCATION = 'https://ai-cpu.lab.bwdemo.io:443/api/embed',
    API_FORMAT = 'ollama',
    MODEL_TYPE = EMBEDDINGS,
    MODEL = 'nomic-embed-text'
);

In [ ]:
CREATE TABLE RandomTexts (
    RandomText NVARCHAR(max),
    embeddings VECTOR(768)
);

In [ ]:
SET NOCOUNT ON;
DECLARE @response NVARCHAR(MAX);

EXEC sp_invoke_external_rest_endpoint
    @url = N'https://openlibrary.org/search.json?q=SQL%20Server',
    @method = 'GET',
    @response = @response OUTPUT;
 
TRUNCATE TABLE RandomTexts
INSERT INTO RandomTexts (RandomText)
SELECT Json_value(value, '$.title') +  isnull (' - ' + Json_value(value, '$.author_name[*]'),'')
FROM   Openjson(Json_query(@response, '$.result.docs'), 'strict $') AS export 

SET NOCOUNT OFF;


In [ ]:
SELECT TOP 5 RandomText FROM RandomTexts

In [ ]:
SELECT COUNT(*) FROM RandomTexts

In [ ]:
update RandomTexts set embeddings = AI_GENERATE_EMBEDDINGS(RandomText,ollama)

In [ ]:
update RandomTexts set embeddings = AI_GENERATE_EMBEDDINGS(RandomText,ollamacpu)

In [ ]:
update RandomTexts set RandomText = RandomText + ' ' + RandomText + ' ' + RandomText

In [ ]:
update RandomTexts set embeddings = AI_GENERATE_EMBEDDINGS(RandomText,ollama)

In [ ]:
update RandomTexts set embeddings = AI_GENERATE_EMBEDDINGS(RandomText,ollamacpu)

In [ ]:
update RandomTexts set RandomText = RandomText + ' ' + RandomText + ' ' + RandomText

In [ ]:
update RandomTexts set embeddings = AI_GENERATE_EMBEDDINGS(RandomText,ollama)

In [ ]:
update RandomTexts set embeddings = AI_GENERATE_EMBEDDINGS(RandomText,ollamacpu)

In [ ]:
SET NOCOUNT ON;
INSERT INTO RandomTexts (RandomText)
SELECT RandomText FROM RandomTexts
INSERT INTO RandomTexts (RandomText)
SELECT RandomText FROM RandomTexts
INSERT INTO RandomTexts (RandomText)
SELECT RandomText FROM RandomTexts
SELECT COUNT(*) FROM RandomTexts
SET NOCOUNT OFF;

In [ ]:
update RandomTexts set embeddings = AI_GENERATE_EMBEDDINGS(RandomText,ollama)

In [ ]:
update RandomTexts set embeddings = AI_GENERATE_EMBEDDINGS(RandomText,ollamacpu)